#   Clase 1 – Fundamentos y diagnóstico de rendimiento
  **Objetivo:** Comprender cómo Spark ejecuta el código y aprender a diagnosticar cuellos de botella.
  
  Estructura:
  1. Arquitectura y ejecución (driver, executors, DAG)
  2. Stages y tasks
  3. Shuffles y data skew
  4. Particiones y paralelismo
  5. Herramientas de diagnóstico


## Bloque 1 – Transformaciones vs Acciones
  Spark es *lazy*: las transformaciones no ejecutan nada hasta que llega una acción.

### Transformaciones
- Operaciones que definen un nuevo DataFrame/RDD.
- _Lazy evaluation_: no ejecutan nada hasta que llega una acción.
- Son operaciones perezosas (_lazy evaluation_): no se ejecutan inmediatamente.
- Devuelven un nuevo DataFrame/RDD con un plan de ejecución actualizado, pero sin lanzar un job.
- Se ejecutan solo cuando se dispara una acción.
- Pueden ser de dos tipos:
    1. Narrow transformations: los datos de una partición se usan solo en esa misma partición.
        - Ejemplos: `map()`, `filter()`, `select()`, `withColumn()`.
        - Más eficientes (no requieren shuffle).
    2. Wide transformations: requieren mover datos entre particiones → shuffle.
        - Ejemplos: `groupBy()`, `join()`, `distinct()`, `repartition()`.
        - Costosas en tiempo y recursos.

In [0]:
from pyspark.sql.functions import col

df = spark.range(1, 100).withColumn("x2", col("id") * 2)  # transformación
df_filtered = df.filter(col("id") > 50)                   # transformación

# Hasta aquí no se ejecuta nada

### Acciones:
- Disparan la ejecución real del plan.
- Devuelven un resultado al driver o escriben datos en almacenamiento.
- Ejemplos:
  - `collect()` → trae todos los datos al driver (⚠️ cuidado con Out Of Memory).
  - `show()` → muestra primeras filas en consola.
  - `count()` → cuenta filas.
  - `write.format("delta").save(...)` → guarda en disco.

In [0]:
df_filtered.show()   # aquí Spark ejecuta el pipeline

### Ejemplo completo

In [0]:
# Transformaciones (lazy)
df = spark.range(1, 1000000)
df2 = df.withColumn("x2", col("id") * 2)   # narrow
df3 = df2.filter(col("x2") % 5 == 0)       # narrow
df4 = df3.groupBy((col("x2") % 10)).count() # wide → shuffle

# Acción (trigger)
df4.show()

Resultado:
- Hasta `.groupBy()` solo hay transformaciones → Spark construye el DAG.
- Al llamar `.show()`, Spark ejecuta el job, lo divide en stages y tasks.

### Spark UI y `explain()`

#### Spark UI
La Spark UI es la interfaz web que te muestra qué está pasando dentro de tu aplicación Spark. En Databricks puedes acceder desde la pestaña Spark UI de cada job o notebook.
Los elementos principales son:
1. Jobs tab
   - Lista de trabajos disparados por acciones (count(), show(), write...).
   - Cada job corresponde a la ejecución de un DAG.
   - Desde aquí entras a Stages.
2. Stages tab
   - Muestra cómo se divide el job en stages (fases).
   - Cada stage tiene múltiples tasks (una por partición).
   - Métricas clave: tiempo, duración, skew, GC, I/O.
3. Tasks tab (dentro de un stage)
   - Detalle de cada partición ejecutada.
   - Puedes detectar data skew: si una task tarda mucho más que las demás.
   - Métricas de input size, shuffle read/write, memoria usada.
4. SQL tab
   - Muy útil si usas DataFrames/SQL.
   - Visualiza el plan lógico y físico en forma de árbol.
   - Identifica dónde ocurren shuffles y scans de tablas. 
5. Storage tab
   - Muestra qué DataFrames/RDDs están cacheados.
   - Permite ver el uso de memoria y disco para persistencia.

En Databricks también tienes el DAG Viewer, un grafo visual que muestra stages y dependencias → muy útil para enseñar.

![Spark UI Jobs](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/SparkUI Jobs.png)

![SparkUI Stages](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/SparkUI Stages.png)

#### `explained()`
El método `.explain()` muestra cómo Spark planea ejecutar tu DataFrame.
Opciones principales:
- `.explain()` → plan físico resumido.
- `.explain("extended")` → plan lógico + optimizado + físico.
- `.explain("cost")` → incluye estimación de costes (filas, bytes).
- `.explain("formatted")` → salida legible en tabla.

### Ejemplo Practico

In [0]:
# Crear DataFrame grande
df = spark.range(0, 10000000)

# Solo definimos transformaciones (lazy)
df_filtered = df.filter(df.id % 2 == 0)
df_transformed = df_filtered.withColumn("id_squared", df.id * df.id)

# Ver plan lógico/físico
df_transformed.explain("formatted")

Paso a paso en el ejemplo:
1. PhotonRange (1)
   - Spark crea un dataset de enteros (de 0 a 10,000,000).
   - Está dividido en 8 splits (= particiones iniciales).
   - Paralelización inicial.
2. PhotonFilter (2)
   - Filtra solo los números pares: (id % 2 = 0).
   - Es un narrow transformation (no hay shuffle).
3. PhotonProject (3)
   - Calcula una nueva columna id_squared.
   - Otra transformación ligera, todavía sin shuffle.
4. PhotonResultStage (4)
   - Prepara los resultados para pasarlos al driver.
5. ColumnarToRow (5)
   - Convierte los datos del formato columnar interno (usado por Photon) a filas normales, porque el driver no entiende el formato columnar.

In [0]:
# Ahora ejecutamos una acción
df_transformed.count()

#### Ejercicio 1
  1. Genera un DataFrame con 10 millones de filas.
  2. Aplica un `filter` y un `groupBy` con `count`.
  3. Usa `.explain("extended")` para inspeccionar el plan.
  
  ❓ Pregunta: ¿en qué momento se ejecuta la query realmente?

In [0]:
# Tu código aquí 👇


#### Explicacion del ejercicio

1. **Parsed Logical Plan**
    ```rust
    'Aggregate ['`%`('id_squared, 10)], [unresolvedalias(...)]
    +- 'Project ...
    +- 'Filter ...
        +- Range ...
    ```
- Aquí Spark acaba de leer tu código (todavía con símbolos `'` y unresolvedalias → significa que no resolvió los nombres ni los tipos).
- Es básicamente un "parseo" del SQL/DataFrame API.
- Es la traducción literal de tu código en objetos internos de Spark.

2. **Analyzed Logical Plan**
    ```rust
    Aggregate [(id_squared#... % cast(10 as bigint))], ...
    +- Project [id, (id * id) AS id_squared]
    +- Filter (id % 2 = 0)
        +- Range (0, 10000000, step=1, splits=Some(8))
    ```
- Ahora Spark ya sabe los tipos de datos y las columnas reales.
- `id_squared` ya no está "unresolved", aparece como `id_squared#11044L`.
- Tu `count(1)` ya está identificado como `bigint`.
- Es la versión "semánticamente válida" del plan.

3. **Optimized Logical Plan**
    ```rust
    Aggregate [_groupingexpression#11046L], ...
    +- Project [((id * id) % 10) AS _groupingexpression#11046L]
    +- Filter (id % 2 = 0)
        +- Range ...
    ```
- Aquí Spark aplica optimizaciones lógicas (Catalyst optimizer).
- Ejemplo: en lugar de calcular id_squared y luego % 10, Spark ya fusionó eso en un solo paso: ((id * id) % 10) → _groupingexpression.
- Esto reduce trabajo intermedio.
- Es un plan más eficiente antes de pasar a lo físico.

4. **Physical Plan**
    ```rust
AdaptiveSparkPlan isFinalPlan=false
+- PhotonGroupingAgg
   +- PhotonShuffleExchangeSource
      +- PhotonShuffleMapStage
         +- PhotonShuffleExchangeSink hashpartitioning(_groupingexpression, 1024)
            +- PhotonGroupingAgg (partial)
               +- PhotonProject ...
                  +- PhotonFilter ...
                     +- PhotonRange ...
    ```
    Aquí está lo bueno
- `PhotonRange` → genera los datos iniciales (0 a 10M).
- `PhotonFilter` → filtra pares.
- `PhotonProject` → calcula `(id*id) % 10`.
- `PhotonGroupingAgg (partial)` → primer aggregation parcial, cada partición cuenta localmente.
- `PhotonShuffleExchangeSink hashpartitioning(..., 1024)` → 🔥 AQUÍ está el shuffle.
- Spark reorganiza los datos en 1024 particiones para agrupar por la clave id_squared % 10.
- `PhotonShuffleMapStage` + `PhotonShuffleExchangeSource` → representan el movimiento de datos entre nodos.
- `PhotonGroupingAgg (final)` → combina los resultados de cada partición tras el shuffle.
- `ColumnarToRow` al final → convierte de formato columnar interno a filas, para que lo entienda el driver.
- Este es el plan real que ejecuta el cluster, y donde verás cuellos de botella (el shuffle).

5. **Photon Explanation**
    ```rust
    The query is fully supported by Photon.
    ```
- Significa que todo el pipeline puede ejecutarse con Photon (engine vectorizado en C++ de Databricks).
- Es mucho más rápido que la ejecución normal en la JVM.
- Si hubiera partes no soportadas, Photon te avisaría.




#### Resumen didactico
- **Parsed plan**: Spark solo entendió la sintaxis.
- **Analyzed plan**: Spark ya entiende las columnas y tipos.
- **Optimized plan**: Spark reorganiza el trabajo para hacerlo más eficiente.
- **Physical plan**: cómo se ejecuta realmente (con shuffle, stages, etc.).
- **Photon explanation**: confirmación de que Photon lo acelera.

## Bloque 2 – DAG, stages y tasks
  Spark divide los pipelines en DAGs → stages → tasks.
  
  Entender esto nos hace comprender cómo afecta al paralelismo y en definitiva al rendimiento.

1. **DAG (Directed Acyclic Graph)**
   - Qué es: Representación del flujo de transformaciones como un grafo dirigido sin ciclos.
    - Cada nodo = operación (map, filter, join, etc.).
    - Cada arista = dependencia de datos entre operaciones.
    - Spark construye el DAG de manera perezosa (lazy evaluation).
    - El DAG no se ejecuta hasta que se encuentra una acción (`.show()`, `.collect()`, `.count()`).

2. **Stages**
   - El DAG se divide en stages según los puntos de shuffle.
   - Narrow dependency: cada partición depende solo de una partición anterior → mismo stage. Ejemplo: `map`, `filter`.
   - Wide dependency: una partición depende de varias → nuevo stage. Ejemplo: `groupBy`, `join`, `distinct`.

3. **Tasks**
   - La unidad más pequeña de trabajo en Spark.
   - Cada task procesa una partición de datos en un executor.
   - Ejemplo: si tengo 200 particiones → Spark lanza 200 tasks (distribuidas en los executors).

### Ejemplo DAG Complejo

In [0]:
from pyspark.sql.functions import col, rand

# Ajustamos las particiones de shuffle para que se generen muchas tasks
spark.conf.set("spark.sql.shuffle.partitions", 200)

# ==============================
# DataFrame base (50 millones filas, 200 particiones iniciales)
# ==============================
df = spark.range(0, 50_000_000, numPartitions=200).withColumn("value", (col("id") * rand()))

# ==============================
# JOB 1: count con 2 stages
# ==============================
# Stage 1: lectura + filtro (narrow)
df_filtered = df.filter(col("value") > 1000)

# Stage 2: shuffle por groupBy + count
df_grouped = df_filtered.groupBy((col("id") % 10).alias("bucket")).count()

# Acción que dispara Job 1
print("Job 1 result:", df_grouped.count())

![](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/Ej1 - Job1.png)

In [0]:
# Cuando llamamos a show(), Spark vuelve a ejecutar el pipeline porque es una acción.
# Esta vez el job tiene 2 stages (lectura + shuffle para mostrar los datos),
# en vez de 3 como en el count, porque show() no necesita contar todas las filas, solo mostrar algunas.

df_grouped.show()

![](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/Ej1 - Job2.png)

In [0]:
# Cuando llamamos a cache(), Spark vuelve a ejecutar el pipeline porque es una acción.
# al igual que antes el job tiene 2 stages pero el segundo más largo con la instrucción de cache al final

df_grouped.cache()


![](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/Ej1 - Job3.png)

In [0]:
# Cuando llamamos a show(), Spark lee de la cache el df y eso ahorra tiempo

df_grouped.show()

![](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/Ej1 - Job4.png)

In [0]:
# ==============================
# JOB 2: join con 3 stages
# ==============================
df1 = spark.range(0, 10_000_000, numPartitions=100).withColumn("k", col("id") % 5000)
df2 = spark.range(0, 20_000_000, numPartitions=150).withColumn("k", col("id") % 5000)

# Stage 1: lectura df1
# Stage 2: lectura df2
# Stage 3: shuffle para join y acción final (show)
df_joined = df1.join(df2, on="k", how="inner")

# Acción que dispara Job 2
df_joined.show(5)

![](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/Ej2 - Job1.png)

![](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/Ej2 - Stage.png)

![](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/Ej2 - Stage2.png)

   **Ejercicio 2**
  1. Generar un DataFrame con `spark.range(0, 10000000)`.
  2. Aplicar transformaciones simples (filter, withColumn).
  3. Comparar dos queries:
     - `df.groupBy("id").count()`
     - `df.filter("id % 2 = 0").count()`

  4. Preguntas:
     - ¿Cuál introduce un shuffle?
     - ¿Cómo se refleja en el `explain("extended")`?
     - ¿En cuál esperas más stages?

In [0]:
# Tu código aquí 👇

## **Ejercicios tipo test**

%md
**1. DAG**

**Pregunta:**
**¿Cuándo se construye el DAG en Spark?**

a) Cuando se arranca el cluster.

b) Cada vez que escribimos una transformación (filter, map, etc.).

c) Solo cuando ejecutamos una acción (show, count, collect).

d) Después de terminar todos los jobs.

%md
**2. Jobs**

**Pregunta:**
**En Spark, ¿qué dispara la creación de un nuevo job?**

a) Un withColumn.

b) Una acción como .show() o .count().

c) Cada vez que se aplica un filtro.

d) Cada vez que se crea un DataFrame.

%md
**3. Stages**

**Pregunta:
¿Qué provoca que Spark divida un job en varios stages?**

a) Cada 1000 filas procesadas.

b) La presencia de un filter.

c) La presencia de una operación con wide dependency (por ejemplo, groupBy, join, orderBy).

d) Cada vez que se seleccionan nuevas columnas.

%md
**4. Tasks**

**Pregunta:
Si un DataFrame tiene 200 particiones y hacemos un count(), ¿cuántas tasks se lanzan en el stage correspondiente?**

a) 1

b) 200

c) Depende de los executors

d) Ninguna

%md
**5. explain()**

**Pregunta:
¿Qué muestra el comando .explain(True) en PySpark?**

a) Solo el resultado de la consulta.

b) El DAG gráfico.

c) El plan lógico (parsed, analyzed, optimized) y el plan físico que Spark planea ejecutar.

d) El número de executors activos en el cluster.

%md
**6. Spark UI**

**Pregunta:
¿Cuál es la diferencia entre abrir la Spark UI desde un notebook o desde la pestaña Compute en Databricks?**

a) Ninguna, siempre se ve lo mismo.

b) Desde el notebook se ve solo la aplicación de ese notebook; desde Compute se ve la lista de todas las aplicaciones Spark activas en el cluster.

c) Desde Compute solo se ven jobs, no stages.

d) Desde el notebook se ven todos los notebooks del cluster.


%md
## **Ejercicios prácticos (predicción de Jobs, Stages y Tasks)**

**1. Count sin shuffle**

In [0]:
df = spark.range(0, 1_000_000, 1, numPartitions=8)
df.count()


%md
Pregunta:

- ¿Cuántos jobs se disparan?
- ¿Cuántos stages tendrá el job principal?
- ¿Cuántas tasks por stage?

%md
***

%md
**2. Filtros encadenados**

In [0]:
df = spark.range(0, 1_000_000, 1, numPartitions=16)
df.filter("id > 1000").filter("id < 900000").count()


%md
Pregunta:

- ¿Qué pasa con los filtros? ¿Se ven varios en el plan físico?
- ¿Cuántos jobs/stages/tasks esperas?

%md
***

%md
**3. Agregación (wide dependency)**

In [0]:
df = spark.range(0, 1_000_000, 1, numPartitions=8)
df.groupBy((df.id % 4).alias("cat")).count().show()


%md
Pregunta:
- ¿Cuántos stages habrá y por qué?
- ¿Cuántas tasks esperas en el reduce? (pista: depende de spark.sql.shuffle.partitions)

%md
***

%md
**4. Order + limit**

In [0]:
df = spark.range(0, 1_000_000, 1, numPartitions=8).withColumn("v", (df.id % 100))
df.orderBy("v").limit(5).show()


%md
Pregunta:

- ¿Cuántos stages aparecen?
- ¿Qué pasa con las particiones en el último stage?

***

%md
**5. Dos acciones consecutivas**

In [0]:
df = spark.range(0, 1_000_000, 1, numPartitions=4)
df.filter("id < 1000").count()
df.filter("id > 500000").show(5)


%md
Pregunta:

- ¿Cuántos jobs en total?
- ¿Se reutilizan stages entre las dos acciones?

## Bloque 3 – Shuffles, Data Skew y Particiones


### Shuffles
  - Redistribución de datos entre particiones.
  - Se producen en `groupBy`, `join`, `distinct`, `orderBy`.
  - Costosos porque implican:
    - Escritura a disco (shuffle files).
    - Transferencia por red.
    - Lectura posterior.
  - ⚠️ Los shuffles son la principal fuente de cuellos de botella en Spark.

### Data Skew (desequilibrio de datos) 
  - Cuando una clave concentra muchos más datos que el resto.
  - Consecuencia: tareas desbalanceadas → unas rápidas, otras muy lentas.
  - Efecto visible: long tail tasks.
  - Ejemplo: 80% de las filas en la misma clave.

### Particiones y paralelismo
  - Spark divide el trabajo en particiones → tasks.
  - Demasiadas particiones → overhead administrativo, demasiados archivos pequeños.
  - Muy pocas particiones → subutilización de CPU, tareas gigantes que bloquean.
  - Config clave:
    - `spark.sql.shuffle.partitions` (por defecto: 200).
    - `repartition()` y `coalesce()`.

  
  El rendimiento depende de encontrar el equilibrio justo.

### Ejemplos

#### Setup particiones

In [0]:
from pyspark.sql.functions import col, when, rand
spark.conf.set("spark.sql.shuffle.partitions", "2000")  # valor inicial por defecto


#### Shuffle grande

In [0]:
spark.conf.set("spark.sql.adaptive.enabled", "false")

In [0]:
# Generamos un DF con 100M filas y 1M claves
df = spark.range(0, 100_000_000).withColumn("key", (col("id") % 1000000))

# Disparar un shuffle con groupBy
res = df.groupBy("key").count()
res.explain("extended")   # 👉 Fíjate en el Exchange hashpartitioning
res.count()


Observa:
- Aparece un Exchange (hashpartitioning) en el plan.
- Esto implica escritura/lectura de shuffle files.

#### Data Skew

In [0]:
from pyspark.sql.functions import col, lit, when

# Forzar muchas particiones en el shuffle
spark.conf.set("spark.sql.shuffle.partitions", 500)

N = 10_000_000
df = spark.range(0, N).withColumn(
    "skewed_key",
    when(col("id") < int(N*0.99), lit(1))   # 95% de las filas van con la clave "1"
    .otherwise((col("id") % 1000) + 2)      # el 5% restante se reparte
)

df_grouped = df.groupBy("skewed_key").count()

df_grouped.show()

![Data Skew KO](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/Data skew ko.png)

Observa:
- Una sola clave (key = 0) concentra la mayoría de las filas.
- En la ejecución, unas tareas terminan rápido y otras tardan muchísimo (long tail).

![Data Skew OK](/Workspace/Users/psanzc@hotmail.com/optimizacion_databricks/Fotos/Data skew ok.png)

#### Ejemplo de ajustes de particiones

In [0]:
# Probar distintos valores de particiones
for p in [10, 200, 2000]:
    spark.conf.set("spark.sql.shuffle.partitions", p)
    print(f"\n=== Shuffle partitions: {p} ===")
    res = df.groupBy("key").count()
    res.count()


- Con 10 → pocas tasks, CPU infrautilizada.
- Con 2000 → overhead, demasiados archivos pequeños.
- Con 200 → equilibrio.

#### Ejercicios

**Ejercicio 1 – Shuffles**
- Usa `explain("extended")` sobre `df.groupBy("key").count()`.
- Identifica dónde aparece el Exchange.
- Cambia el número de particiones (`spark.sql.shuffle.partitions`) y repite.
- Pregunta: ¿cuántos stages aparecen con 10, 200 y 2000 particiones?

In [0]:
# Tu codigo aqui


**Ejercicio 2 – Skew en acción**
- Genera un DataFrame con skew (80% de los datos en la misma clave).
- Ejecuta `groupBy("key").count()` y mide tiempos.
- Mira en el plan físico → ¿cómo se refleja el shuffle?
- Pregunta: ¿qué pasa con las tareas asociadas a la clave dominante?

In [0]:
# Tu codigo aqui


Para comparar sin skew:

In [0]:
# Tu codigo aqui


## Ejercicio Final

Contexto: eres el equipo de datos de una aerolínea. Cada noche calculáis KPIs operativos sobre 100M vuelos. El informe llega tarde. Tenéis que diagnosticar y optimizar el pipeline de “retraso medio por aeropuerto y compañía”.

El objetivo es optimizar este trozo de código para optimizar el proceso:

In [0]:
from pyspark.sql.functions import rand, expr, col

N = 10_000_000_000
flights = (spark.range(N)
           .withColumn("airport", expr("CASE WHEN rand() < 0.7 THEN 'JFK' WHEN rand() < 0.15 THEN 'LAX' ELSE 'ORD' END"))
           .withColumn("carrier", expr("CASE id % 5 WHEN 0 THEN 'AA' WHEN 1 THEN 'UA' WHEN 2 THEN 'DL' WHEN 3 THEN 'SW' ELSE 'IB' END"))
           .withColumn("delay_min", (rand()*180).cast("int")))

# Versión inicial (no optimizada)
kpis0 = flights.groupBy("airport","carrier").avg("delay_min")
kpis0.explain("extended")


Qué esperar: Exchange `hashpartitioning((airport, carrier), N)`, y tiempo alto por skew en `JFK`.

Paso 1. Lo primero es diagnosticar

In [0]:
# Comprobar distribución

# Probar distintos suhffle partitions

Paso 2. Optimizar

In [0]:
# Reparticionar por clave antes de agregar


Paso 3. Responder a estas preguntas
 - ¿Dónde estaba el shuffle? ¿Cómo lo viste?
 - ¿Cómo detectaste el skew sin Spark UI?
 - ¿Qué valor de spark.sql.shuffle.partitions te dio mejor resultado en tu clúster?
 - ¿Cuánto mejoró el tiempo con repartition/salting? (anotar segundos antes/después).